In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import psycopg2

In [2]:
# Make a reference to the books.csv file path
csv_path = "Data/append.csv"

# Import the books.csv file as a DataFrame
append_df = pd.read_csv(csv_path, encoding="utf-8")
append_df

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant,average
0,1,3,10,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489
1,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221
2,1,49,9991,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582
3,1,51,4,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902
4,1,55,11,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016
...,...,...,...,...,...,...,...,...,...
1949,72,53,3,Puerto Rico,NaN,18.383983,-65.618888,PM2.5 - Local Conditions,6.868067
1950,72,57,12,Puerto Rico,Guayama,17.955378,-66.162122,PM2.5 - Local Conditions,5.303571
1951,72,61,5,Puerto Rico,Guaynabo,18.440095,-66.114460,PM2.5 - Local Conditions,7.487179
1952,72,113,4,Puerto Rico,Ponce,18.009558,-66.627249,PM2.5 - Local Conditions,6.285841


## Group by Pollutant and State

In [3]:
group_df = append_df.groupby(["pollutant", "state"])

In [6]:
# Get mean values of pollutants
group_df['average'].describe()

count      mean       std       min  \
pollutant                state                                                
Ozone                    Alabama         22.0  0.041233  0.003035  0.036283   
                         Alaska           2.0  0.028852  0.007056  0.023862   
                         Arizona         54.0  0.047179  0.002688  0.039959   
                         Arkansas         8.0  0.039501  0.002096  0.035541   
                         California     175.0  0.043045  0.007191  0.020109   
...                                       ...       ...       ...       ...   
PM2.5 - Local Conditions Virginia        18.0  6.897253  0.515078  6.252101   
                         Washington       3.0  7.805554  1.624766  5.937624   
                         West Virginia   13.0  8.042143  0.946669  6.986555   
                         Wisconsin        2.0  7.361398  0.919348  6.711321   
                         Wyoming         14.0  3.833899  1.483481  1.830063   

                                             25%       50%       75%       max  
pollutant                state                                                  
Ozone                    Alabama        0.038518  0.041874  0.043869  0.045025  
                         Alaska         0.026357  0.028852  0.031346  0.033841  
                         Arizona        0.045834  0.046974  0.049249  0.052386  
                         Arkansas       0.038822  0.039985  0.040498  0.042022  
                         California     0.038322  0.042986  0.048684  0.060326  
...                                          ...       ...       ...       ...  
PM2.5 - Local Conditions Virginia       6.527238  6.766625  7.172250  7.980342  
                         Washington     7.262756  8.587889  8.739520  8.891150  
                         West Virginia  7.270270  7.771698  8.833913  9.599145  
                         Wisconsin      7.036359  7.361398  7.686437  8.011475  
                         Wyoming        2.833525  3.747745  4.212231  7.655556  

[106 rows x 8 columns]

In [ ]:
group_df['average'].describe().to_csv("group_pollutant_state.csv")

In [ ]:
group_df['average'].describe().to_json("group_pollutant_state.json")

## Make ozone mean and pm2.5 mean for each state separately to merge them

In [7]:
# Select ozone
append_ozone = append_df.loc[append_df['pollutant'] == 'Ozone']
append_ozone

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant,average
0,1,3,10,Alabama,Fairhope,30.497478,-87.880258,Ozone,0.040489
1,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,Ozone,0.038221
2,1,49,9991,Alabama,Crossville,34.289001,-85.970065,Ozone,0.044582
3,1,51,4,Alabama,Wetumpka,32.535680,-86.255193,Ozone,0.040902
4,1,55,11,Alabama,Southside,33.904039,-86.053867,Ozone,0.044016
...,...,...,...,...,...,...,...,...,...
1282,56,45,3,Wyoming,NaN,43.873056,-104.191944,Ozone,0.045297
1283,72,21,10,Puerto Rico,Bayamon,18.420089,-66.150615,Ozone,0.013117
1284,72,33,8,Puerto Rico,Catano,18.440774,-66.126531,Ozone,0.009864
1285,72,97,7,Puerto Rico,Mayaguez,18.216038,-67.144107,Ozone,0.005862


In [8]:
#  Group by states
append_ozone = append_ozone.groupby(["state"])
append_ozone['average'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
Alabama,22.0,0.041233,0.003035,0.036283,0.038518,0.041874,0.043869,0.045025
Alaska,2.0,0.028852,0.007056,0.023862,0.026357,0.028852,0.031346,0.033841
Arizona,54.0,0.047179,0.002688,0.039959,0.045834,0.046974,0.049249,0.052386
Arkansas,8.0,0.039501,0.002096,0.035541,0.038822,0.039985,0.040498,0.042022
California,175.0,0.043045,0.007191,0.020109,0.038322,0.042986,0.048684,0.060326
Colorado,34.0,0.046275,0.003571,0.038922,0.045208,0.046723,0.048658,0.055680
Connecticut,12.0,0.044233,0.002754,0.038413,0.043195,0.044770,0.046296,0.047632
Country Of Mexico,1.0,0.045291,NaN,0.045291,0.045291,0.045291,0.045291,0.045291
Delaware,7.0,0.043504,0.002509,0.038480,0.042955,0.044652,0.044973,0.045541


In [9]:
append_ozone['average'].describe().to_csv("ozone_state_df.csv")

In [10]:
# Select pm
append_pm = append_df.loc[append_df['pollutant'] == 'PM2.5 - Local Conditions']
append_pm

,State Code,County Code,Site Num,state,city,latitude,longitude,pollutant,average
1287,1,3,10,Alabama,Fairhope,30.497478,-87.880258,PM2.5 - Local Conditions,7.551402
1288,1,27,1,Alabama,Ashland,33.284928,-85.803608,PM2.5 - Local Conditions,7.550467
1289,1,33,1002,Alabama,Muscle Shoals,34.762619,-87.638097,PM2.5 - Local Conditions,7.213235
1290,1,49,1003,Alabama,Crossville,34.288567,-85.969858,PM2.5 - Local Conditions,7.550000
1291,1,55,10,Alabama,Gadsden,33.991494,-85.992647,PM2.5 - Local Conditions,8.410526
...,...,...,...,...,...,...,...,...,...
1949,72,53,3,Puerto Rico,NaN,18.383983,-65.618888,PM2.5 - Local Conditions,6.868067
1950,72,57,12,Puerto Rico,Guayama,17.955378,-66.162122,PM2.5 - Local Conditions,5.303571
1951,72,61,5,Puerto Rico,Guaynabo,18.440095,-66.114460,PM2.5 - Local Conditions,7.487179
1952,72,113,4,Puerto Rico,Ponce,18.009558,-66.627249,PM2.5 - Local Conditions,6.285841


In [11]:
#  Group by states
pm_state_df = append_pm.groupby(["state"])
pm_state_df['average'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
Alabama,17.0,8.318954,0.839212,7.213235,7.551402,8.302679,8.705000,10.310833
Alaska,9.0,9.072469,2.704737,6.076667,7.427439,8.377059,9.796793,15.206647
Arizona,7.0,6.478174,3.023702,1.881818,4.929968,6.892437,7.670037,11.372951
Arkansas,13.0,8.532442,0.770124,7.524490,8.007500,8.278151,8.827273,10.339831
California,79.0,8.504724,2.603402,3.141667,6.595507,8.398529,10.864407,13.752975
Colorado,13.0,5.856631,1.941265,2.500000,4.721053,6.016102,6.828814,9.488525
Connecticut,8.0,5.605916,0.909548,3.800833,5.025798,6.060834,6.248195,6.378947
Delaware,5.0,6.955217,0.589111,6.246667,6.501266,6.955446,7.488119,7.584589
District Of Columbia,8.0,8.231982,0.385972,7.868539,7.950335,8.082998,8.426000,8.871350


In [ ]:
pm_state_df['average'].describe().to_csv("pm_state_df.csv")

## Bring back ozone_state and pm_state again to merge them

In [13]:
# Make a reference to the books.csv file path
ozone_csv_path = "Data/ozone_state_df.csv"

# Import the books.csv file as a DataFrame
ozone_mean_df = pd.read_csv(ozone_csv_path, encoding="utf-8")
ozone_mean_df

,state,count,mean,std,min,25%,50%,75%,max
0,Alabama,22.0,0.041233,0.003035,0.036283,0.038518,0.041874,0.043869,0.045025
1,Alaska,2.0,0.028851,0.007056,0.023862,0.026357,0.028851,0.031346,0.033841
2,Arizona,54.0,0.047179,0.002688,0.039959,0.045834,0.046974,0.049249,0.052386
3,Arkansas,8.0,0.039501,0.002096,0.035541,0.038822,0.039985,0.040498,0.042022
4,California,175.0,0.043045,0.007191,0.020109,0.038321,0.042986,0.048685,0.060326
5,Colorado,34.0,0.046275,0.003571,0.038922,0.045208,0.046723,0.048658,0.055680
6,Connecticut,12.0,0.044233,0.002754,0.038413,0.043195,0.044769,0.046296,0.047632
7,Country Of Mexico,1.0,0.045291,NaN,0.045291,0.045291,0.045291,0.045291,0.045291
8,Delaware,7.0,0.043504,0.002509,0.038480,0.042955,0.044652,0.044973,0.045541
9,District Of Columbia,3.0,0.042532,0.003907,0.039082,0.040411,0.041739,0.044257,0.046775


In [14]:
# Make a reference to the books.csv file path
pm_csv_path = "Data/pm_state_df.csv"

# Import the books.csv file as a DataFrame
pm_mean_df = pd.read_csv(pm_csv_path, encoding="utf-8")
pm_mean_df

,state,count,mean,std,min,25%,50%,75%,max
0,Alabama,17.0,8.318954,0.839212,7.213235,7.551402,8.302679,8.705000,10.310833
1,Alaska,9.0,9.072469,2.704737,6.076667,7.427439,8.377059,9.796793,15.206647
2,Arizona,7.0,6.478174,3.023702,1.881818,4.929968,6.892437,7.670037,11.372951
3,Arkansas,13.0,8.532442,0.770124,7.524490,8.007500,8.278151,8.827273,10.339831
4,California,79.0,8.504724,2.603402,3.141667,6.595507,8.398529,10.864406,13.752975
5,Colorado,13.0,5.856631,1.941265,2.500000,4.721053,6.016102,6.828814,9.488525
6,Connecticut,8.0,5.605916,0.909548,3.800833,5.025798,6.060834,6.248195,6.378947
7,Delaware,5.0,6.955217,0.589111,6.246667,6.501266,6.955446,7.488119,7.584589
8,District Of Columbia,8.0,8.231982,0.385972,7.868539,7.950335,8.082998,8.426000,8.871350
9,Florida,16.0,6.501624,0.689204,5.304167,5.920204,6.549786,7.009511,7.599167


In [15]:
# Drop off unnecessary columns
ozone_mean_df = ozone_mean_df[["state", "count", "mean", "min", "max"]]
ozone_mean_df

,state,count,mean,min,max
0,Alabama,22.0,0.041233,0.036283,0.045025
1,Alaska,2.0,0.028851,0.023862,0.033841
2,Arizona,54.0,0.047179,0.039959,0.052386
3,Arkansas,8.0,0.039501,0.035541,0.042022
4,California,175.0,0.043045,0.020109,0.060326
5,Colorado,34.0,0.046275,0.038922,0.055680
6,Connecticut,12.0,0.044233,0.038413,0.047632
7,Country Of Mexico,1.0,0.045291,0.045291,0.045291
8,Delaware,7.0,0.043504,0.038480,0.045541
9,District Of Columbia,3.0,0.042532,0.039082,0.046775


In [16]:
# Drop off unnecessary columns
pm_mean_df = pm_mean_df[["state", "count", "mean", "min", "max"]]
pm_mean_df

,state,count,mean,min,max
0,Alabama,17.0,8.318954,7.213235,10.310833
1,Alaska,9.0,9.072469,6.076667,15.206647
2,Arizona,7.0,6.478174,1.881818,11.372951
3,Arkansas,13.0,8.532442,7.524490,10.339831
4,California,79.0,8.504724,3.141667,13.752975
5,Colorado,13.0,5.856631,2.500000,9.488525
6,Connecticut,8.0,5.605916,3.800833,6.378947
7,Delaware,5.0,6.955217,6.246667,7.584589
8,District Of Columbia,8.0,8.231982,7.868539,8.871350
9,Florida,16.0,6.501624,5.304167,7.599167


In [17]:
# Merge mean dfs
merge_mean_df = pd.merge(ozone_mean_df, pm_mean_df, on=["state"], how="outer")
merge_mean_df

,state,count_x,mean_x,min_x,max_x,count_y,mean_y,min_y,max_y
0,Alabama,22.0,0.041233,0.036283,0.045025,17.0,8.318954,7.213235,10.310833
1,Alaska,2.0,0.028851,0.023862,0.033841,9.0,9.072469,6.076667,15.206647
2,Arizona,54.0,0.047179,0.039959,0.052386,7.0,6.478174,1.881818,11.372951
3,Arkansas,8.0,0.039501,0.035541,0.042022,13.0,8.532442,7.524490,10.339831
4,California,175.0,0.043045,0.020109,0.060326,79.0,8.504724,3.141667,13.752975
5,Colorado,34.0,0.046275,0.038922,0.055680,13.0,5.856631,2.500000,9.488525
6,Connecticut,12.0,0.044233,0.038413,0.047632,8.0,5.605916,3.800833,6.378947
7,Country Of Mexico,1.0,0.045291,0.045291,0.045291,NaN,NaN,NaN,NaN
8,Delaware,7.0,0.043504,0.038480,0.045541,5.0,6.955217,6.246667,7.584589
9,District Of Columbia,3.0,0.042532,0.039082,0.046775,8.0,8.231982,7.868539,8.871350


In [19]:
#Rename columns
merge_mean_df = merge_mean_df.rename(columns={
    "mean_x": "ozone_mean", "count_x": "count_ozone","mean_x": "mean_ozone","min_x": "min_ozone", "max_x": "max_ozone",
    "mean_y": "pm_mean", "count_y": "count_pm","mean_y": "mean_pm","min_y": "min_pm", "max_y": "max_pm",
})
merge_mean_df

,state,count_ozone,mean_ozone,min_ozone,max_ozone,count_pm,mean_pm,min_pm,max_pm
0,Alabama,22.0,0.041233,0.036283,0.045025,17.0,8.318954,7.213235,10.310833
1,Alaska,2.0,0.028851,0.023862,0.033841,9.0,9.072469,6.076667,15.206647
2,Arizona,54.0,0.047179,0.039959,0.052386,7.0,6.478174,1.881818,11.372951
3,Arkansas,8.0,0.039501,0.035541,0.042022,13.0,8.532442,7.524490,10.339831
4,California,175.0,0.043045,0.020109,0.060326,79.0,8.504724,3.141667,13.752975
5,Colorado,34.0,0.046275,0.038922,0.055680,13.0,5.856631,2.500000,9.488525
6,Connecticut,12.0,0.044233,0.038413,0.047632,8.0,5.605916,3.800833,6.378947
7,Country Of Mexico,1.0,0.045291,0.045291,0.045291,NaN,NaN,NaN,NaN
8,Delaware,7.0,0.043504,0.038480,0.045541,5.0,6.955217,6.246667,7.584589
9,District Of Columbia,3.0,0.042532,0.039082,0.046775,8.0,8.231982,7.868539,8.871350


In [20]:
merge_mean_df.to_csv("merge_mean_state.csv")

In [30]:
merge_mean_df.to_json('merge_mean.json')

## Send csv files to sqlite

In [21]:
from sqlalchemy import create_engine

In [22]:
# Connect to the database
engine = create_engine('sqlite:///aqi.sqlite', echo=False)
# I also created a db file:
# engine = create_engine('sqlite:///aqi.db', echo=False)
sqlite_connection = engine.connect()

In [23]:
# Adding the base DF (I think this would give it its own table?)
merge_mean_df.to_sql('merged_state_mean', con=engine)

In [24]:
# Adding the base DF (I think this would give it its own table?)
ozone_mean_df.to_sql('ozone_state_mean', con=engine)

In [25]:
# Adding the base DF (I think this would give it its own table?)
pm_mean_df.to_sql('pm_state_mean', con=engine)

In [26]:
# Confirming tables are in the DB
engine.execute("SELECT * FROM merged_state_mean").fetchall()

[(0, 'Alabama', 22.0, 0.04123281818181818, 0.036283, 0.045025, 17.0, 8.318953882352941, 7.213235000000001, 10.310833),
 (1, 'Alaska', 2.0, 0.0288515, 0.023862, 0.033841, 9.0, 9.072468888888887, 6.076667, 15.206646999999998),
 (2, 'Arizona', 54.0, 0.04717894444444443, 0.039958999999999995, 0.052386, 7.0, 6.478173571428571, 1.881818, 11.372951),
 (3, 'Arkansas', 8.0, 0.0395005, 0.035541, 0.042022000000000004, 13.0, 8.532442230769231, 7.524489999999999, 10.339831),
 (4, 'California', 175.0, 0.04304492571428573, 0.020109000000000002, 0.060326, 79.0, 8.50472353164557, 3.141667, 13.752975),
 (5, 'Colorado', 34.0, 0.0462750588235294, 0.038922000000000005, 0.05567999999999999, 13.0, 5.856631076923076, 2.5, 9.488525),
 (6, 'Connecticut', 12.0, 0.04423341666666667, 0.038413, 0.047632, 8.0, 5.605915875000001, 3.8008330000000004, 6.378947),
 (7, 'Country Of Mexico', 1.0, 0.045291000000000005, 0.045291000000000005, 0.045291000000000005, None, None, None, None),
 (8, 'Delaware', 7.0, 0.0435042857142

In [27]:
# Confirming tables are in the DB
engine.execute("SELECT * FROM ozone_state_mean").fetchall()

[(0, 'Alabama', 22.0, 0.04123281818181818, 0.036283, 0.045025),
 (1, 'Alaska', 2.0, 0.0288515, 0.023862, 0.033841),
 (2, 'Arizona', 54.0, 0.04717894444444443, 0.039958999999999995, 0.052386),
 (3, 'Arkansas', 8.0, 0.0395005, 0.035541, 0.042022000000000004),
 (4, 'California', 175.0, 0.04304492571428573, 0.020109000000000002, 0.060326),
 (5, 'Colorado', 34.0, 0.0462750588235294, 0.038922000000000005, 0.05567999999999999),
 (6, 'Connecticut', 12.0, 0.04423341666666667, 0.038413, 0.047632),
 (7, 'Country Of Mexico', 1.0, 0.045291000000000005, 0.045291000000000005, 0.045291000000000005),
 (8, 'Delaware', 7.0, 0.04350428571428572, 0.03848, 0.045541000000000005),
 (9, 'District Of Columbia', 3.0, 0.042531999999999986, 0.039082, 0.046775),
 (10, 'Florida', 58.0, 0.03693505172413794, 0.032443, 0.04034),
 (11, 'Georgia', 19.0, 0.042614947368421065, 0.038905, 0.048131),
 (12, 'Hawaii', 2.0, 0.028130500000000003, 0.027407, 0.028854),
 (13, 'Idaho', 4.0, 0.0372035, 0.027357, 0.045691),
 (14, 'Illi

In [28]:
# Confirming tables are in the DB
engine.execute("SELECT * FROM pm_state_mean").fetchall()

[(0, 'Alabama', 17.0, 8.318953882352941, 7.213235000000001, 10.310833),
 (1, 'Alaska', 9.0, 9.072468888888887, 6.076667, 15.206646999999998),
 (2, 'Arizona', 7.0, 6.478173571428571, 1.881818, 11.372951),
 (3, 'Arkansas', 13.0, 8.532442230769231, 7.524489999999999, 10.339831),
 (4, 'California', 79.0, 8.50472353164557, 3.141667, 13.752975),
 (5, 'Colorado', 13.0, 5.856631076923076, 2.5, 9.488525),
 (6, 'Connecticut', 8.0, 5.605915875000001, 3.8008330000000004, 6.378947),
 (7, 'Delaware', 5.0, 6.9552174, 6.246667, 7.5845889999999985),
 (8, 'District Of Columbia', 8.0, 8.23198225, 7.868538999999997, 8.87135),
 (9, 'Florida', 16.0, 6.5016239375, 5.304167, 7.599167),
 (10, 'Georgia', 19.0, 8.239873842105263, 6.032143, 9.516529),
 (11, 'Hawaii', 14.0, 2.257113357142857, -1.006569, 3.4133129999999996),
 (12, 'Idaho', 9.0, 7.711405666666666, 3.927479, 10.070809),
 (13, 'Illinois', 20.0, 8.994996800000001, 7.542609, 11.235714),
 (14, 'Indiana', 29.0, 8.878872344827586, 7.405556, 11.947826),
 (1

In [29]:
# All the table names in the sqlite file (aqi.sqlite)
engine.table_names()

['merged_state_mean', 'ozone_state_mean', 'pm_state_mean']